# Test applying a model

In [1]:
# Load model and tokenizer

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_dirpath = '/home/mamille3/white_supremacist_lang/output/bert/6topic_annotated_5x/checkpoint-21210'

tokenizer = AutoTokenizer.from_pretrained(model_dirpath)
model = AutoModelForSequenceClassification.from_pretrained(model_dirpath)

In [4]:
# Preprocess text and apply tokenizer
import pandas as pd
from datasets import Dataset

example_data = [
    'this flower smells really nice',
    'white genocide is real',
]
example_df = pd.DataFrame({'text': example_data})
example_df

,text
0,this flower smells really nice
1,white genocide is real


In [3]:
def preprocess(df):
    return tokenizer(df["text"], truncation=True)

dataset = Dataset.from_pandas(example_df)
tokenized = dataset.map(preprocess, batched=True)

Parameter 'function'=<function preprocess at 0x7f2daf54e4c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
# Classify (as just a PyTorch Module)
inputs = tokenizer(example_data[0], return_tensors='pt')
# Should do for multiple examples (map it to the list)
# inputs = list(map(tokenizer(example_data, return_tensors='pt')
output = model(**inputs)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [7]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.6068, -4.1664]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [5]:
# Classify (with Trainer)
import numpy as np

output = model.predict(test_tokenized)

# Binary predictions
preds = np.argmax(output.predictions, axis=-1)
preds

AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'predict'

In [ ]:
# Save out numeric class probability predictions
prob_outpath = os.path.join(self.output_dir, f'{dataset}_pred_probs.txt')
prob = scipy.special.softmax(pred_output.predictions, axis=-1)
class_prob = pd.DataFrame(prob)
class_prob.columns = class_prob.columns.map(self.id2label)
class_prob.to_json(prob_outpath, orient='records', lines=True